In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
## dataSet setting
churn = pd.read_csv('C:/Users/KIHyuk/Documents/GitHub/Jupyter_Repo/Data/churn.csv',sep=',',header=0)

In [3]:
churn.columns = [heading.lower() for heading in churn.columns.str.replace(' ','_').str.replace("\'","").str.strip('?')]

## np.where(조건문, true변경값, false변경값)
churn['churn01'] = np.where(churn['churn'] == 'True.', 1., 0.) 

churn.head()

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn,churn01
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,16.78,244.7,91,11.01,10.0,3,2.70,1,False.,0.0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,16.62,254.4,103,11.45,13.7,3,3.70,1,False.,0.0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,10.30,162.6,104,7.32,12.2,5,3.29,0,False.,0.0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,5.26,196.9,89,8.86,6.6,7,1.78,2,False.,0.0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,12.61,186.9,121,8.41,10.1,3,2.73,3,False.,0.0


In [4]:
## churn 에 따른 그룹화

## 그룹별 기술통계
churn.groupby(['churn'])[['day_charge','eve_charge','night_charge','intl_charge','account_length','custserv_calls']].agg(['count','mean','std'])

day_charge                       eve_charge                       \
            count       mean        std      count       mean       std   
churn                                                                     
False.       2850  29.780421   8.530835       2850  16.918909  4.274863   
True.         483  35.175921  11.729710        483  18.054969  4.396762   

       night_charge                     intl_charge                      \
              count      mean       std       count      mean       std   
churn                                                                     
False.         2850  9.006074  2.299768        2850  2.743404  0.751784   
True.           483  9.235528  2.121081         483  2.889545  0.754152   

       account_length                       custserv_calls                      
                count        mean       std          count      mean       std  
churn                                                                           
False.           2850  100.793684  39.88235           2850  1.449825  1.163883  
True.             483  102.664596  39.46782            483  2.229814  1.853275

In [6]:
## 변수별 통계량
print(churn.groupby(['churn']).agg({'day_charge' : ['mean', 'std'],
                             'eve_charge' : ['mean','std'],
                             'night_charge' : ['mean','std'],
                             'intl_charge' : ['mean','std'],
                             'account_length' : ['count','min','max'],
                             'custserv_calls' : ['count','min','max']}))

       day_charge            eve_charge           night_charge            \
             mean        std       mean       std         mean       std   
churn                                                                      
False.  29.780421   8.530835  16.918909  4.274863     9.006074  2.299768   
True.   35.175921  11.729710  18.054969  4.396762     9.235528  2.121081   

       intl_charge           account_length          custserv_calls          
              mean       std          count min  max          count min max  
churn                                                                        
False.    2.743404  0.751784           2850   1  243           2850   0   8  
True.     2.889545  0.754152            483   1  225            483   0   9  


In [12]:
## total_charges(새로운 변수) 기준 그룹화 + 그룹별 통계량

churn['total_charges'] = churn['day_charge'] + churn['eve_charge'] + churn['night_charge'] + churn['intl_charge']

factor_cut = pd.cut(churn.total_charges,5,precision=2) ## cut, total_charges 변수의 데이터를 폭이 같은 5개 구간으로 나눔 

def get_stats(group): ## 그룹 통계량 반환 함수
    return {'min' : group.min(), 'max' : group.max(),
           'count' : group.count(), 'mean' : group.mean(),
           'std' : group.std()}

grouped = churn.custserv_calls.groupby(factor_cut)

print(grouped.apply(get_stats).unstack())

                 count  max      mean  min       std
total_charges                                       
(22.86, 37.57]    70.0  5.0  1.528571  0.0  1.348337
(37.57, 52.22]   742.0  7.0  1.564690  0.0  1.305234
(52.22, 66.86]  1726.0  9.0  1.581692  0.0  1.326646
(66.86, 81.51]   735.0  9.0  1.523810  0.0  1.295209
(81.51, 96.15]    60.0  5.0  1.516667  0.0  1.359108
